In [30]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [55]:
data_dir = '/lustre/groups/epigenereg01/workspace/projects/vale/MLM/'

In [60]:
model = 'PhyloP241' #PhyloP100 or PhyloP241

In [24]:
utr_table = pd.read_csv(data_dir + 'UTR_coords/GRCh38_3_prime_UTR_clean-sorted.bed', sep='\t',
                       header = None, names=['chrom','seq_start','seq_end','seq_name','strand'], usecols=[0,1,2,3,5]).set_index('seq_name')

In [32]:
phylop_res = pd.read_csv(data_dir + '../tools/PhyloP/PhyloP100_PhyloP241_intersect.tsv.gz', sep=' ',
                       header = None, names=['chrom','pos','PhyloP100','PhyloP241']).sort_values(by=['chrom','pos']).set_index('chrom')

In [53]:
phylop_res.pos = phylop_res.pos-1 #to 0-based

In [61]:
phylop_scores = {}

for seq_name, (chrom, start_seq,end_seq,strand) in tqdm(utr_table.iterrows(),total=len(utr_table)):
    L = end_seq-start_seq #sequence length
    phylop_scores[seq_name] = np.full((L,), np.nan)
    df = phylop_res.loc[chrom]
    start_idx = np.searchsorted(df.pos,start_seq)
    end_idx = np.searchsorted(df.pos,end_seq)
    for _, row in df.iloc[start_idx:end_idx].iterrows():
        phylop_scores[seq_name][int(row.pos)-start_seq] = row[model]
    if strand=='-':
        phylop_scores[seq_name] = phylop_scores[seq_name][::-1] #reverse scores on negative strand to be compatible with other models

100%|██████████| 18178/18178 [14:21<00:00, 21.09it/s]


In [62]:
with open(data_dir + f'PhyloP/{model}_3UTR.pickle', 'wb') as f:
    pickle.dump(phylop_scores,f)